In [5]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import pandas as pd
import csv
import time


In [3]:
# Ask for the search query
search_query = "tv" 

# Base URL for Amazon
base_url = 'https://www.amazon.in/'

# Number of pages to scrape
num_pages = 2  

# Initializing the WebDriver with Chrome
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))

# Create a CSV file and write headers
csv_file = open('amazon_products.csv', 'w', newline='', encoding='utf-8')
csv_writer = csv.writer(csv_file)
csv_writer.writerow(['Product Name', 'Price', 'Rating', 'Number of Ratings', 'Past Month Bought'])

# Scraping data from multiple pages
for page in range(1, num_pages + 1):
    # Update URL for each page
    page_url = f"{base_url}?page={page}"  
    driver.get(page_url)
    # Adjust this delay as needed for the page to load
    time.sleep(3)  

    # Find the search input field and enter the search query
    search_input = driver.find_element_by_id('twotabsearchtextbox')
    search_input.clear()
    search_input.send_keys(search_query)

    # Find and click the search button
    search_button = driver.find_element_by_id('nav-search-submit-button')
    search_button.click()
    # Wait for the search results to load
    time.sleep(3)  

    # Get the page source
    page_source = driver.page_source

    # Parse the page source using BeautifulSoup
    soup = BeautifulSoup(page_source, 'html.parser')

    product_cards = soup.find_all('div', {'data-component-type': 's-search-result'})

    for card in product_cards:
        # Product Name
        product_name = card.find('span', {'class': 'a-size-medium a-color-base a-text-normal'})
        if product_name:
            product_name = product_name.text.strip()
        else:
            product_name = 'Not available'

        # Price
        price = card.find('span', {'class': 'a-price-whole'})
        if price:
            price = price.text.strip()
        else:
            price = 'Not available'

        # Rating
        rating = card.find('span', {'class': 'a-icon-alt'})
        if rating:
            rating = rating.text.split()[0]
        else:
            rating = 'Not available'

        # Number of Ratings
        num_ratings = card.find('span', {'class': 'a-size-base'})
        if num_ratings:
            num_ratings = num_ratings.text.split()[0]
            if num_ratings=="M.R.P:":
                num_ratings== 'Not available'
        else:
            num_ratings = 'Not available'
            
    # past_month_bought
        past_month_bought = card.find('span', {'class': 'a-size-base a-color-secondary'})
        if past_month_bought:
            past_month_bought = past_month_bought.text.strip()
            if past_month_bought=="M.R.P:":
                past_month_bought= 'Not available'
        else:
            past_month_bought = 'Not available'

        # Write data to CSV file
        csv_writer.writerow([product_name, price, rating, num_ratings, past_month_bought])

# Close the CSV file
csv_file.close()

# Close the WebDriver
driver.quit()


<ipython-input-3-721630fddcaa>:27: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  search_input = driver.find_element_by_id('twotabsearchtextbox')
<ipython-input-3-721630fddcaa>:32: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  search_button = driver.find_element_by_id('nav-search-submit-button')


In [7]:
amazon_products=pd.read_csv("amazon_products.csv")
amazon_products.head()

,Product Name,Price,Rating,Number of Ratings,Past Month Bought
0,LG 80 cm (32 inches) HD Ready Smart LED TV 32L...,"12,990",4.3,"13,984",1K+ bought in past month
1,Acer 80 cm (32 inches) Advanced I Series HD Re...,"11,999",4.1,"3,910",1K+ bought in past month
2,Hisense 305 cm (120 inches) Trichrom ALR Scree...,"4,99,999",3.4,3,Not available
3,Redmi 80 cm (32 inches) F Series HD Ready Smar...,"11,499",4.2,"57,844",5K+ bought in past month
4,LG 80 cm (32 inches) HD Ready Smart LED TV 32L...,"14,490",4.3,"13,984",1K+ bought in past month
